We implemented K-means with map reduce. We used the value for number of cluster given by the Elbow Method found in the exercise below.

In [36]:
import numpy as np
from numpy import linalg
from pyspark.ml.linalg import Vectors

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

We read the data

In [12]:
data = sc.textFile('s3://sophja.k/data_score.csv')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
points=data.map(lambda row : row.split(","))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
row=points.take(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
row

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[['-0.0988635239706018', '1.0547428686872697', '-0.55669363138511', '1.5686337566928372'], ['-0.004175630616080824', '-0.4217818953437049', '-0.2694882987311586', '-0.00861670843399599'], ['-0.20044036082777814', '0.24184749652546683', '-1.2788218830625382', '-0.008175224309425234'], ['-0.16939056186668425', '0.14613804714676096', '-0.6333258381538082', '-1.4021344078547264'], ['0.11525730794485466', '-0.4783662705597016', '0.382540962226797', '-1.4027253587065533'], ['-0.37713548513359124', '-0.4666029690052431', '-0.28267419545048084', '-0.0082052170381025'], ['0.0749583647011032', '1.1140699526351527', '-1.1687423395641279', '-0.010008525871680593'], ['0.5382149164178932', '-0.44008677676958174', '0.36249484059837445', '1.5693174787635467'], ['-0.35340573931017727', '-0.33994121408299777', '2.4918167594430374', '-1.3904884103682387'], ['0.11588756688699053', '-0.8800119943039049', '3.543928348182661', '-1.3911921787827586']]

In [16]:
points=points.map(lambda x: np.array(x, dtype=np.float32))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
initial_centroids = points.takeSample(False, 4)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

we save as first centroids the ones we find with takeSample

In [18]:
centroids_broadcast = sc.broadcast(initial_centroids)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

we define the function distance that calculates the distance between the points and centroides or the old and new centroids 

In [19]:
def distance(p,q):
    dist=linalg.norm(p - q, 2)
    return dist

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

we define a function assign_centroids that assign the new centroid to each point every time the new centroid is calculated

In [20]:
def assign_centroids(p):
    min_dist = float("inf")
    centroids = centroids_broadcast.value
    nearest_centroid = 0
    for i in range(len(centroids)):
        distance_  = distance(centroids[i], p)
        if(distance_ < min_dist):
            min_dist = distance_
            nearest_centroid = i
    
    return (nearest_centroid, p)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

The final function creates the centroid assign the definitive cluster to each points based on the distance they have to the centroid

In [22]:

n=0
while True:
    cluster_assignment_rdd = points.map(assign_centroids)
    
    average_rdd = cluster_assignment_rdd.reduceByKey(lambda x,y: np.mean([x,y], axis=0)).sortByKey(ascending=True)
    new_centroids = [item[1] for item in average_rdd.collect()]
    #we check if the value of the old and new centroids are similiar
    old_centroids = centroids_broadcast.value

    for i in range(len(old_centroids)):
        check = distance(old_centroids[i],new_centroids[i]) <= 0.05 #check if there are values less then threshold

    n += 1
    #repeat if n is less then the max number of iteration  or distance more then threshold
    if(check.any() == False and n < 10):
        centroids_broadcast = sc.broadcast(new_centroids)
    else:
        break

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
row=cluster_assignment_rdd.take(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
row

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[(1, array([-0.09886353,  1.0547428 , -0.5566936 ,  1.5686338 ], dtype=float32)), (1, array([-0.00417563, -0.4217819 , -0.2694883 , -0.00861671], dtype=float32)), (1, array([-0.20044036,  0.2418475 , -1.2788218 , -0.00817522], dtype=float32)), (2, array([-0.16939056,  0.14613804, -0.6333258 , -1.4021344 ], dtype=float32)), (2, array([ 0.11525731, -0.47836626,  0.38254097, -1.4027253 ], dtype=float32)), (1, array([-0.3771355 , -0.46660298, -0.2826742 , -0.00820522], dtype=float32)), (1, array([ 0.07495836,  1.1140699 , -1.1687423 , -0.01000853], dtype=float32)), (1, array([ 0.5382149 , -0.44008678,  0.36249483,  1.5693175 ], dtype=float32)), (2, array([-0.35340574, -0.3399412 ,  2.4918168 , -1.3904884 ], dtype=float32)), (2, array([ 0.11588757, -0.880012  ,  3.5439284 , -1.3911922 ], dtype=float32))]

In [37]:
dff = cluster_assignment_rdd.map(lambda x: (int(x[0]), Vectors.dense(x[1:])))

mydf = spark.createDataFrame(dff,schema=["'n_cluster'", "features"])



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…